# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
# MODEL = "gpt-4o-mini"
# openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
google_api_key = os.getenv('GOOGLE_API_KEY')
MODEL = "gemini-2.0-flash-exp"
openai = OpenAI(base_url='https://generativelanguage.googleapis.com/v1beta/openai/', api_key=google_api_key)


OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [5]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [6]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [7]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [8]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [12]:
def confirm_booking(destination_city, price):
    print(f"Tool confirm_booking called for {destination_city} at {price}")
    return True

In [14]:
confirm_booking("Berlin","$499")

Tool confirm_booking called for Berlin at $499


True

In [15]:
# There's a particular dictionary structure that's required to describe our function:

booking_function = {
    "name": "confirm_booking",
    "description": "Confirm booking on a return ticket to the destination city. Call this whenever you need to book the ticket, for example when a customer asks 'Book a return ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },"price": {
                "type": "string",
                "description": "This is the fare of the travel",
            }            
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [16]:
tools = [{"type": "function", "function": price_function},{"type": "function", "function": booking_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [21]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    print('finish_reason:', response.choices[0].finish_reason)
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print('message:', message)
        response, city, price = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools = tools)
        if response.choices[0].finish_reason=="tool_calls":
            message = response.choices[0].message
            print('message:', message)
            response= book_tool_call(message, city, price)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [23]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    print('tool_call:',tool_call)
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    print("destination_City: ",city)
    price = get_ticket_price(city)
    print("price: ",price)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city, price

In [20]:
# We have to write that function handle_tool_call:

def book_tool_call(message, city, price):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    result = confirm_booking(city, price)
    response = {
        "role": "tool",
        "content": json.dumps({"result": True}),
        "tool_call_id": tool_call.id
    }
    return response

In [24]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


finish_reason: stop
finish_reason: tool_calls
message: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='', function=Function(arguments='{"destination_city":"Tokyo"}', name='get_ticket_price'), type='function')])
tool_call: ChatCompletionMessageToolCall(id='', function=Function(arguments='{"destination_city":"Tokyo"}', name='get_ticket_price'), type='function')
destination_City:  Tokyo
Tool get_ticket_price called for Tokyo
price:  $1400
finish_reason: tool_calls
message: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='', function=Function(arguments='{"price":"$1400","destination_city":"Tokyo"}', name='confirm_booking'), type='function')])
tool_call: ChatCompletionMessageToolCall(id='', function=Function(arguments='{"price":"$1400","destination_city":"Tokyo"}', name='confirm_booking'), type='fun